In [ ]:
from pathlib import Path

from hbn_pl.io import load_spe,  validate_spe_data
from hbn_pl.preprocess import (
    remove_frames,
    remove_cosmic,
    background_subtract,
    average_and_normalise,
)
from hbn_pl.peaks import (
    find_peaks_pl,
    identify_zpl_psb,
    classify_emitter,
)
from hbn_pl.energy import energy_offset_mev
from hbn_pl.plot import plot_spectrum, plot_energy

current_dir = Path.cwd()
DATA_DIR = current_dir / "Data/Bad emitters"
FILE_NAME = '2025-11-17 16_22_14 sample5_DCMd2_200uw_emitter4 2182.spe'

OUTPUT_DIR = Path("outputs/sample7_emitter01")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# Need to also be able to save metadata and where the emitters are, perferably in a json file 

# Need to figure out how to do this automatically...
FRAMES_TO_REMOVE = []     # e.g. [3, 7]
COSMIC_FRAMES = []        # e.g. [12, 24]
BG_SLICE = (10, 200)

spe_path = DATA_DIR / FILE_NAME
wavelength, frames = load_spe(spe_path)

validate_spe_data(spe_path, wavelength, frames) # -> need to put this in the loading 

frames = remove_frames(frames, FRAMES_TO_REMOVE)
frames = remove_cosmic(frames, COSMIC_FRAMES)
frames = background_subtract(frames, bg_slice=BG_SLICE)

avg, avg_norm = average_and_normalise(frames)

peaks, properties = find_peaks_pl(
    wavelength,
    avg_norm,
    height=0.1,
    prominence=0.7,
)

zpl, psb = identify_zpl_psb(peaks, wavelength, avg_norm)
classification = classify_emitter(peaks, wavelength) # Probably wrong here, can you get more emitters?

print("Emitter classification:", classification)

if zpl is None:
    raise RuntimeError("No ZPL found — cannot continue energy analysis")

print(f"ZPL: {zpl['wl']:.2f} nm, I = {zpl['I']:.3f}")
if psb is not None:
    print(f"PSB: {psb['wl']:.2f} nm, I = {psb['I']:.3f}")
    print("ZPL / PSB ratio:", zpl["I"] / psb["I"])

energy_mev = energy_offset_mev(wavelength, zpl["wl"])

plot_spectrum(
    wavelength,
    avg_norm,
    peaks=peaks,
    zpl=zpl,
    psb=psb,
    outpath=OUTPUT_DIR / "spectrum_wavelength.png",
)

plot_energy(
    energy_mev,
    avg_norm,
    energy_window=(-20, 200),
    outpath=OUTPUT_DIR / "spectrum_energy.png",
)

print("Analysis complete.")
print(f"Plots saved to: {OUTPUT_DIR.resolve()}")


NameError: name '__file__' is not defined